https://comp.probspace.com/competitions/religious_art/discussions/Oregin-Post2a43024f366d7a2a8a05

In [3]:
!git clone https://github.com/ultralytics/yolov5 > /dev/null

Cloning into 'yolov5'...
remote: Enumerating objects: 9185, done.
remote: Total 9185 (delta 0), reused 0 (delta 0), pack-reused 9185
Receiving objects: 100% (9185/9185), 9.63 MiB | 8.24 MiB/s, done.
Resolving deltas: 100% (6373/6373), done.


In [4]:
import sys

sys.path.append("..")

In [33]:
import os
from dataclasses import dataclass
from glob import glob
from typing import *

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import timm
import torch
import torch.nn as nn
from sklearn.model_selection import StratifiedKFold
from torch import optim
from torch.optim import lr_scheduler
from torch.utils import data
from torch.utils.data import DataLoader
from torchvision import transforms as T
from tqdm import tqdm

%matplotlib inline

In [150]:
run_name = "exp034"

@dataclass
class InputPath:
    _prefix: str = "../input"
    train_images: str = f"{_prefix}/christ-train-imgs.npz"
    train_labels: str = f"{_prefix}/christ-train-labels.npz"
    test_images: str = f"{_prefix}/christ-test-imgs.npz"
        
        
@dataclass
class OutputPath:
    _prefix: str = "../output"
    logs: str = f"{_prefix}/logs"
    model: str = f"{_prefix}/model"
    submission: str = f"{_prefix}/submission"
     
        
def load_npz(path: str) -> np.array:
    return np.load(path)["arr_0"]


def generate_kf() -> Generator:
    kf = StratifiedKFold(
        n_splits=5,
        shuffle=True,
        random_state=42,
    )
    return kf

In [8]:
train_images = load_npz(InputPath.train_images)
test_images = load_npz(InputPath.test_images)

In [20]:
!mkdir -p images

In [21]:
from PIL import Image


for i, image in tqdm(enumerate(train_images), total=len(train_images)):
    if i < 10:
        filename = f"00{i}"
    elif i < 100:
        filename = f"0{i}"
    else:
        filename = i
    pil_image = Image.fromarray(image)
    pil_image.save(f"images/train_{filename}.jpg")

for i, image in tqdm(enumerate(test_images), total=len(test_images)):
    if i < 10:
        filename = f"00{i}"
    elif i < 100:
        filename = f"0{i}"
    else:
        filename = i
    pil_image = Image.fromarray(image)
    pil_image.save(f"images/test_{filename}.jpg")

100%|██████████| 497/497 [00:00<00:00, 796.21it/s]


In [31]:
!cd yolov5 && python detect.py --weights yolov5x.pt --img 224 --conf 0.05 --source /workspace/notebooks/images --save-txt --save-conf

detect: weights=['yolov5x.pt'], source=/workspace/notebooks/images, imgsz=[224, 224], conf_thres=0.05, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 v5.0-405-gfad57c2 torch 1.8.1+cu102 CUDA:0 (GeForce RTX 2080 Ti, 11016.9375MB)

Fusing layers... 
Model Summary: 476 layers, 87730285 parameters, 0 gradients
image 1/1151 /workspace/notebooks/images/test_000.jpg: 224x224 1 bed, Done. (0.013s)
image 2/1151 /workspace/notebooks/images/test_001.jpg: 224x224 Done. (0.012s)
image 3/1151 /workspace/notebooks/images/test_002.jpg: 224x224 13 persons, 2 ties, Done. (0.013s)
image 4/1151 /workspace/notebooks/images/test_003.jpg: 224x224 1 clock, Done. (0.013s)
image 5/1151 /workspace/notebooks/images/test_004.jpg: 224x224 Done. (0.0

In [93]:
# 検出結果のtxtから、検出した物体の数をカウント
results = []
for file_path in tqdm(sorted(glob('./yolov5/runs/detect/exp/labels/*.txt'))):
    file_id = file_path.split('/')[-1].split('.')[0]
    file_num = file_id.split('_')[-1]
    with open(file_path, 'r') as f:
        x = f.read().replace('\n', ' ').strip().split(' ')
    x = np.array(x).astype(np.float32).reshape(-1, 6)
    label, counts = np.unique(x[:, 0], return_counts=True)
    d = {}
    d['index'] = file_id
    for i in range(len(label)):
        d[int(label[i])] = counts[i]
    results.append(d)

100%|██████████| 811/811 [00:00<00:00, 16024.93it/s]


In [110]:
file_id, file_num

('train_651', '651')

In [121]:
df = pd.DataFrame(results)
df = df.set_index("index")
df = df.T.sort_index().T.fillna(0).astype(int)
df = df.reset_index()
df.head()

,index,0,1,2,3,4,6,7,8,11,...,68,69,71,72,73,74,75,76,77,79
0,test_000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,test_002,13,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,test_003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,test_005,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,test_006,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [127]:
train = df[df["index"].apply(lambda x: x.startswith("train_"))].reset_index(drop=True)
test = df[df["index"].apply(lambda x: x.startswith("test_"))].reset_index(drop=True)

In [148]:
train_index_set = set(train["index"].apply(lambda x: int(x.split("_")[1])).tolist())
new_train_index_list = []
for i in range(len(train_images)):
    if i not in train_index_set:
        if i < 10:
            filename = f"train_00{i}"
        elif i < 100:
            filename = f"train_0{i}"
        else:
            filename = f"train_{i}"
        new_train_index_list.append(filename)
        
test_index_set = set(test["index"].apply(lambda x: int(x.split("_")[1])).tolist())
new_test_index_list = []
for i in range(len(test_images)):
    if i not in test_index_set:
        if i < 10:
            filename = f"test_00{i}"
        elif i < 100:
            filename = f"test_0{i}"
        else:
            filename = f"test_{i}"
        new_test_index_list.append(filename)
        
train = pd.concat([train, pd.DataFrame({"index": new_train_index_list})], axis=0, ignore_index=True).fillna(0)
train = train.sort_values("index").reset_index(drop=True).astype({c: int for c in train.columns if c != "index"})
test = pd.concat([test, pd.DataFrame({"index": new_test_index_list})], axis=0, ignore_index=True).fillna(0)
test = test.sort_values("index").reset_index(drop=True).astype({c: int for c in test.columns if c != "index"})

display(train.head())
display(test.head())

,index,0,1,2,3,4,6,7,8,11,...,68,69,71,72,73,74,75,76,77,79
0,train_000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,train_001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,train_002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,train_003,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,train_004,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,index,0,1,2,3,4,6,7,8,11,...,68,69,71,72,73,74,75,76,77,79
0,test_000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,test_001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,test_002,13,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,test_003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,test_004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [149]:
train.to_csv(f"{InputPath._prefix}/train_yolov5.csv", index=False)
test.to_csv(f"{InputPath._prefix}/test_yolov5.csv", index=False)

---

In [152]:
train_labels = load_npz(InputPath.train_labels)
train_df = pd.DataFrame({"target": train_labels})

kf = generate_kf()
kf_generator = kf.split(train_df, train_df["target"])
for fold_i, (_, val_idx) in enumerate(kf_generator):
    train_df.loc[val_idx, "fold"] = fold_i
train_df = train_df.assign(fold=train_df["fold"].astype(int))
train_df.head()

,target,fold
0,5,0
1,11,3
2,8,2
3,2,1
4,6,0


In [153]:
train_df = pd.concat((train_df, train), axis=1)
train_df.head()

,target,fold,index,0,1,2,3,4,6,7,...,68,69,71,72,73,74,75,76,77,79
0,5,0,train_000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11,3,train_001,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,2,train_002,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,1,train_003,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,0,train_004,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [156]:
fold = 0
X_tr = train_df[train_df["fold"]!=fold].drop(["target", "fold", "index"], axis=1).values
y_tr = train_df[train_df["fold"]!=fold]["target"].values
X_val = train_df[train_df["fold"]==fold].drop(["target", "fold", "index"], axis=1).values
y_val = train_df[train_df["fold"]==fold]["target"].values

In [176]:
import lightgbm as lgb


lgbm_params = {
    "boosting_type": "gbdt",
    "tree_learner": "serial",
    "objective": "multiclass",
    "metric": "multi_logloss",
    "n_jobs": 8,
    "n_estimators": 10000,
    "seed": 42,
    "verbose": -1,
    "learning_rate": 0.02,
    "num_leaves": 127,
    "max_depth": -1,
    "min_child_samples": 40,
    "subsample": 0.9,
    "subsample_freq": 3,
    "colsample_bytree": 0.9,
    "reg_alpha": 0.1,
    "reg_lambda": 1.0,
}

model = lgb.LGBMClassifier(**lgbm_params)
model.fit(X_tr, y_tr, eval_set=[(X_tr, y_tr), (X_val, y_val)], early_stopping_rounds=100, verbose=-1)
y_pred = model.predict(X_val)
score = np.sum(y_val == y_pred) / len(y_val)

print(f"accuracy: {score:.3f}")

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[42]	training's multi_logloss: 2.34489	valid_1's multi_logloss: 2.41864
accuracy: 0.206
